In [1]:
!pip install -q llama-index
!pip install pypdf
!pip install google-generativeai
!pip install transformers
!pip install pinecone-client
!pip install sentence_transformers
!pip install -q transformers einops accelerate langchain bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.8/15.8 MB 61.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.9/147.9 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 57.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
tensorflow-probability 0.22.0 requires typing-extensions<4.6.0, but you have typing-extensions 4.9.0 which is incompatible.


In [3]:
!pip install langchain

In [3]:
from llama_index.llms import OpenRouter
from llama_index.llms import ChatMessage

llm = OpenRouter(
    api_key="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",
    max_tokens=256,
    context_window=4096,
    model="mistralai/mixtral-8x7b-instruct",
)

In [4]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

embed_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


.gitattributes:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [5]:
from llama_index import ServiceContext
from llama_index import set_global_service_context
service_context = ServiceContext.from_defaults(
    llm = llm,
    embed_model = embed_model,
    chunk_size=256,
)

set_global_service_context(service_context)

In [8]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, StorageContext



In [24]:
!mkdir -p 'data/wms/'
!wget 'https://en.wikipedia.org/wiki/Warehouse_management_system' -O 'data/wms/wms.txt'

import os
os.makedirs('data/wms/', exist_ok=True)

documents = SimpleDirectoryReader("./data/wms").load_data()
os.environ['PINECONE_API_KEY'] = '69cce988-1524-4dd4-bcf1-8221a8b5a576'
# environment is found next to API key in the console
os.environ['PINECONE_ENVIRONMENT'] = 'gcp-starter'

import os
from pinecone import Pinecone, ServerlessSpec
from llama_index.vector_stores import PineconeVectorStore

pinecone = Pinecone(
        api_key=os.environ['PINECONE_API_KEY'],
    environment=os.environ['PINECONE_ENVIRONMENT']
    )

# initialize connection to pinecone

pinecone_index = pinecone.Index('quickstart')

wms_vector_store = PineconeVectorStore(
    pinecone_index=pinecone_index, namespace="wikipedia_info"
)

# allow the PineconeVectorStore to be used as storage
storage_context = StorageContext.from_defaults(vector_store=wms_vector_store)


# allow the creation of an Index
wms_vector_index = VectorStoreIndex.from_documents(documents,
                                       storage_context=storage_context)
query_engine = wms_vector_index.as_query_engine()

--2024-01-21 21:04:22--  https://en.wikipedia.org/wiki/Warehouse_management_system
Resolving en.wikipedia.org (en.wikipedia.org)... 103.102.166.224, 2001:df2:e500:ed1a::1
Connecting to en.wikipedia.org (en.wikipedia.org)|103.102.166.224|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 95172 (93K) [text/html]
Saving to: ‘data/wms/wms.txt’

data/wms/wms.txt    100%[===================>]  92.94K  --.-KB/s    in 0.004s  

2024-01-21 21:04:22 (21.0 MB/s) - ‘data/wms/wms.txt’ saved [95172/95172]



Upserted vectors:   0%|          | 0/406 [00:00<?, ?it/s]

In [25]:
!pip install pyngrok
!pip install flask_cors

In [26]:
from flask import Flask
from pyngrok import ngrok

portno = 8000

In [27]:
cd /content/drive/MyDrive/Colab Notebooks/Flask-model

/content/drive/MyDrive/Colab Notebooks/Flask-model


In [28]:
from flask import Flask, request,render_template

In [29]:
import requests


In [30]:
from flask import jsonify
from flask_cors import CORS


In [32]:
app = Flask(__name__)
CORS(app)

ngrok.set_auth_token('2ZGr4Gf5m7wNAUvDwHdgdSSZJPX_4mGbGzSCpfAFuc7fPpGGi')
public_url = ngrok.connect(portno).public_url



# Allow the creation of an Index

wikipedia_data_dir = '/content/drive/MyDrive/Colab Notebooks/Flask-model/data/wiki'
def download_wikipedia_content(url):
    response = requests.get(url)
    if response.status_code == 200:
        # Save the content to a file
        with open(os.path.join(wikipedia_data_dir, 'wikipedia_content.txt'), 'w', encoding='utf-8') as file:
            file.write(response.text)

def load_data_into_vector_store():
    # Load data into PineconeVectorStore
    documents = SimpleDirectoryReader(wikipedia_data_dir).load_data()
    wms_vector_index = VectorStoreIndex.from_documents(documents,
                                       storage_context=storage_context)

def train_query_engine_with_data():
    # Train the query engine with loaded data
    query_engine = wms_vector_index.as_query_engine()
@app.route("/")
def home():
  return render_template("chat.html")
@app.route('/prompt/<input_text>', methods=['GET'])
def get_response(input_text):
    # Call your query_engine function with the input_text
    bot_response = query_engine.query(input_text)

    # Prepare the response data
    response_data = {
        'messages': [
            {'content': input_text}
        ],
        'candidates': [
            {'content': bot_response.response}
        ]
    }

    # Return the response as JSON
    return response_data
@app.route("/get", methods=["GET", "POST"])
def chat():
    msg = request.form["msg"]
    input = msg
    bot_response = query_engine.query(input)

    return bot_response.response
@app.route('/train_query_engine', methods=['POST','GET'])
def train_query_engine():
    wikipedia_url = request.form.get('wikipedia_url')
    print(wikipedia_url)
    # Create the directory if it doesn't exist
    os.makedirs(wikipedia_data_dir, exist_ok=True)
    # Download Wikipedia content based on the provided URL
    download_wikipedia_content(wikipedia_url)
    load_data_into_vector_store()
    train_query_engine_with_data()
    # Return a response indicating that training is complete
    return jsonify({'status': 'Training complete'})


print(f"to access go to {public_url}")
app.run(port=portno)

to access go to https://4637-34-124-183-98.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:8000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 21:08:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 21:08:05] "GET /static/style.css HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 21:08:07] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 21:08:23] "POST /get HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [21/Jan/2024 21:08:50] "POST /get HTTP/1.1" 200 -
